# LangGraph + MCP + GPT-5 for Banking and Finance

This notebook demonstrates integration of **Model Context Protocol (MCP)** with **LangGraph** and **GPT‑5** to build tool‑calling AI workflows for **Banking & Finance**.


## 🧩 What is Model Context Protocol (MCP)
**MCP** is an open standard that defines how Large Language Models communicate with external tools and APIs.

- Discover available tools (`list_tools`)
- Call tools dynamically (`call_tool`)
- Receive structured JSON responses

Think of MCP as the **USB protocol for AI tools** — plug & play interoperability for any LLM.

## 🤔 Why MCP ?
| Traditional Integration | With MCP |
|--------------------------|----------|
| Hard-coded APIs | Dynamic tool discovery |
| App-specific contracts | Standardized JSON-RPC |
| Tight coupling | Sandboxed isolation |
| One LLM per API | Multi-LLM interoperability |

**Benefits**
- 🔌 Interoperable
- 🔒 Secure and sandboxed
- 🧠 Auto-discovery of tools
- ⚙️ Works with LangGraph, Copilot, Claude

## ⚙️ MCP Architecture
```
GPT‑5 ↔ LangGraph ↔ MCP Client ↔ MCP Server ↔ Business Tools
```
**Server** → hosts logic (e.g., EMI calculator)

**Client** → bridges LangGraph ↔ Server

**LLM Agent** → GPT‑5 decides which tool to call

## 🛠️ Environment Setup

In [ ]:
!pip install -U mcp langchain-mcp langchain-openai langgraph

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-your-key-here"  # Replace with valid key

## 🧱 Basic MCP Server Example

In [1]:
%%writefile mcp_server.py

from mcp.server.fastmcp import FastMCP

mcp = FastMCP("BankingMCP")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

@mcp.tool()
def message_of_the_day() -> str:
    return "Stay curious. Build boldly."

if __name__ == "__main__":
    mcp.run()

Overwriting mcp_server.py


In [ ]:
!python mcp_server.py

## 🧪 MCP Client Test

In [3]:
%%writefile mcp_client_test.py

import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def test_client():
    params = StdioServerParameters(command="python", args=["mcp_server.py"])
    async with stdio_client(params) as (r, w):
        async with ClientSession(r, w) as session:
            await session.initialize()
            tools = await session.list_tools()
            print("Available tools:", [t[0] for t in tools])
            result = await session.call_tool("add", {"a": 10, "b": 20})
            print("Result:", result.structuredContent.get("result"))

asyncio.run(test_client())

Writing mcp_client_test.py


In [ ]:
!python mcp_client_test.py

[10/31/25 22:25:29] INFO     Processing request of type            ]8;id=985938;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=300098;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             ListToolsRequest                                   
Available tools: ['meta', 'nextCursor', 'tools']
                    INFO     Processing request of type            ]8;id=955175;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=81691;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             CallToolRequest                                    
Result: 30


## 🤖 LangGraph + GPT‑5 Integration

In [ ]:
%%writefile langchain_mcp_integration.py

import asyncio, warnings
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp import MCPToolkit
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
warnings.filterwarnings("ignore", category=UserWarning)

async def run_agent():
    params = StdioServerParameters(command="python", args=["mcp_server.py"])
    async with stdio_client(params) as (r, w):
        async with ClientSession(r, w) as session:
            await session.initialize()
            toolkit = MCPToolkit(session=session)
            await toolkit.initialize()
            tools = toolkit.get_tools()
            print("Loaded tools:", [t.name for t in tools])

            llm = ChatOpenAI(model="gpt-5", temperature=0, request_timeout=30)
            agent = create_react_agent(llm, tools)
            result = await agent.ainvoke({
                "messages":[("user","Add 40 and 60 then give message of the day.")]
            })
            print("💬 Response:", result["messages"][-1].content)

asyncio.run(run_agent())

Overwriting langchain_mcp_integration.py


In [9]:
!python langchain_mcp_integration.py

[10/31/25 22:28:07] INFO     Processing request of type            ]8;id=643788;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=884471;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             ListToolsRequest                                   
✅ Loaded tools: ['add', 'message_of_the_day']
[10/31/25 22:28:17] INFO     Processing request of type            ]8;id=8500;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=875747;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             CallToolRequest                                    
                    INFO     Processing request of type            ]8;id=699554;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=376926;file:///opt/anacon

## 💰 Demo 1 — Loan EMI Calculator (MCP Server)

In [10]:
%%writefile mcp_server_emi.py
import math
from mcp.server.fastmcp import FastMCP
mcp = FastMCP("LoanEMIServer")

@mcp.tool()
def calculate_emi(principal: float, rate: float, tenure_years: float) -> float:
    """Calculate monthly EMI."""
    r = rate / (12 * 100)
    n = tenure_years * 12
    emi = principal * r * ((1 + r)**n) / ((1 + r)**n - 1)
    return round(emi, 2)

@mcp.tool()
def message_of_the_day() -> str:
    return "Dream big — finance smartly 💡"

if __name__ == "__main__":
    mcp.run()

Writing mcp_server_emi.py


### 🧮 EMI Client + LangGraph Integration

In [11]:
%%writefile mcp_client_emi.py

import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp import MCPToolkit
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

async def run_emi_agent():
    params = StdioServerParameters(command="python", args=["mcp_server_emi.py"])
    async with stdio_client(params) as (r, w):
        async with ClientSession(r, w) as session:
            await session.initialize()
            toolkit = MCPToolkit(session=session)
            await toolkit.initialize()
            tools = toolkit.get_tools()
            print("Tools:", [t.name for t in tools])

            llm = ChatOpenAI(model="gpt-5", temperature=0)
            agent = create_react_agent(llm, tools)
            result = await agent.ainvoke({
                "messages":[("user","Calculate EMI for ₹5 L loan @ 10% for 3 years and share message of the day.")]
            })
            print("Agent Response:", result["messages"][-1].content)

asyncio.run(run_emi_agent())

Writing mcp_client_emi.py


In [12]:
!python mcp_client_emi.py

[10/31/25 22:30:27] INFO     Processing request of type            ]8;id=119933;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=970310;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             ListToolsRequest                                   
Tools: ['calculate_emi', 'message_of_the_day']
[10/31/25 22:30:33] INFO     Processing request of type            ]8;id=453181;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=840959;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             CallToolRequest                                    
                    INFO     Processing request of type            ]8;id=98476;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=578;file:///opt/anacond

## 🧮 Demo 2 — Risk Scoring Tool (MCP Server)

In [13]:
%%writefile mcp_server_risk.py

from mcp.server.fastmcp import FastMCP
mcp = FastMCP("RiskServer")

@mcp.tool()
def calculate_risk_score(age: int, income: float, liabilities: float) -> str:
    """Return basic financial risk score."""
    ratio = liabilities / income if income else 1
    score = max(0, 100 - (age*0.2 + ratio*50))
    if score > 75: return "Low Risk"
    elif score > 40: return "Moderate Risk"
    else: return "High Risk"

@mcp.tool()
def message_of_the_day() -> str:
    return "Plan your finances before they plan you 💼"

if __name__ == "__main__":
    mcp.run()

Writing mcp_server_risk.py


### 🧠 Risk Scoring Client + LangGraph

In [14]:
%%writefile mcp_client_risk.py

import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp import MCPToolkit
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

async def run_risk_agent():
    params = StdioServerParameters(command="python", args=["mcp_server_risk.py"])
    async with stdio_client(params) as (r, w):
        async with ClientSession(r, w) as session:
            await session.initialize()
            toolkit = MCPToolkit(session=session)
            await toolkit.initialize()
            tools = toolkit.get_tools()
            print("Tools:", [t.name for t in tools])

            llm = ChatOpenAI(model="gpt-5", temperature=0)
            agent = create_react_agent(llm, tools)
            result = await agent.ainvoke({
                "messages":[("user","Calculate risk score for age = 35, income = 15 L INR, liabilities = 3 L INR.")]
            })
            print("Agent Response:", result["messages"][-1].content)

asyncio.run(run_risk_agent())

Writing mcp_client_risk.py


In [15]:
!python mcp_client_risk.py

[10/31/25 22:32:06] INFO     Processing request of type            ]8;id=139469;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=63451;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             ListToolsRequest                                   
Tools: ['calculate_risk_score', 'message_of_the_day']
[10/31/25 22:32:11] INFO     Processing request of type            ]8;id=300616;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=487186;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             CallToolRequest                                    
Agent Response: Your financial risk score is: Low Risk.

Inputs considered:
- Age: 35
- Income: ₹15,00,000 per year
- Liabilities: ₹3,00,000

If you’d like, I can break down how this score is calculated 

## 🔧 Future Demos (Placeholders)
- Account Balance Calculator
- Credit Card Reward Estimator
- Transaction Analyzer (Alerts)
- Natural Language Banking Query Agent

## ✅ Summary
- MCP standardizes tool calling between LLMs and code.
- LangGraph adds stateful multi‑tool reasoning.
- GPT‑5 acts as the controller to invoke banking tools intelligently.

## 📚 References
- [ModelContextProtocol.io](https://modelcontextprotocol.io)
- [LangGraph Docs](https://langchain-ai.github.io/langgraph/)
- [LangChain MCP GitHub](https://github.com/rectalogic/langchain-mcp)
- [OpenAI GPT‑5 Docs](https://platform.openai.com/docs)
